# 6. Feladatsor: megoldások

*(Lagrange- és Hermite-interpoláció)*

### P1. Feladat

Írjunk programot, ami adott $x$ és $y$ -pontokat és az ott felveendő értékeket tartalmazó- vektorokra interpolációs polinomot illeszt az 1. feladatbeli módszerrel. Számoljuk ki a rendszert leíró mátrix kondíciószámát, ha $x=[2,3,6,10,14]$ és $y=[1,-2,4,9,13]$.

In [ ]:
import numpy as np

def poly_fit(xs: np.array, ys: np.array, n=None):
    # xs: alappontok, m hosszú vektor
    # ys: alappontokba óhajtott értékek, m hosszú vektor
    # n: fokszám, nemnegatív egész (n=m esetén interpoláció, n<m esetén regresszió)
    if n is None:
        n=np.size(xs)
            
    nn=np.arange(n)
    xs=xs[:,None]
    nn=nn[None,:]
    A = xs**nn 
    b = ys
    coeff=np.linalg.solve(A.T@A,A.T@b) # ha n<m, akkor normálegyenlet, négyzetes, invertálható A esetén A^*Ax=A^*b megoldása ugyanaz mint, Ax=b megoldása 
    coeff=coeff[::-1]
    return A, coeff

In [ ]:
A,coeff=poly_fit( np.array([2,3,6,10,14]),np.array([1,-2,4,9,13]))
np.linalg.cond(A)

In [ ]:
def poly_f(c,x): #mint az np.polyval
    #c: együtthatók
    #x: változó
    n=np.size(c)
    nn=np.arange(n-1,-1,-1)
    xx=x**nn
    return np.sum(c*xx)

In [ ]:
poly_f([1,0,0,0],2)

In [ ]:
poly_f(coeff,14)

### P2. Feladat

Írjunk programot, ami adott $x$ és $y$ -pontokat és az ott felveendő értékeket tartalmazó- vektorokra interpolációs polinomot illeszt a Newton-alak használatával, majd egy további $z$ pontban kiértékeli azt.

In [ ]:

def poly_fit_newton(xs, ys):
    def p(k):
        if k == 0:
            return lambda z: ys[0]
        else:
            c = (ys[k] - p(k-1)(xs[k])) / np.prod(xs[k] - xs[0:k])
            return lambda z: p(k-1)(z) + c * np.prod(z - xs[0:k])

    n = min(len(xs), len(ys)) - 1
    return p(n)

# Példa használat
xs = np.array([1, 2, 3])
ys = np.array([1, 4, 9])

interpolation_poly = poly_fit_newton(xs, ys)

# Kiértékeljük az interpolációs polinomot egy adott pontban
x = 2.5
print("Az interpolációs polinom értéke x =", x, "pontban:", interpolation_poly(x))


### P3. Feladat
Tekintsük az $$f(x) = \frac{1}{1 + 25x^2}$$ függvényt a $[-1, 1]$ intervallumon. Írjunk programot, ami adott $n$ esetén  az intervallum egy $n+1$ elemű, egyenletes rácsán felvett értékek alapján $n$-edfokú polinomot illeszt erre a függvényre. 

Ábrázoljuk a kapott polinomokat $n=2,4,8,12$ esetén. Mit tapasztalunk?

In [ ]:
def f_fv(x):
    return 1 / (1 + 25*x**2)

In [ ]:
import matplotlib.pyplot as plt

xx = np.linspace(-1,1,128)
ax = plt.axes()
for n in [2,4,8,12]:
    xs=np.linspace(-1,1,n)
    poly=poly_fit_newton(xs,f_fv(xs))
    poly_v=np.vectorize(poly)
    ax.plot(xx, poly_v(xx), label="n=" + str(n))


ax.plot(xx, f_fv(xx), label="függvény")    

ax.legend()
ax.set_xlabel(r"x")
ax.set_ylabel(r"y")

*Megjegyzés:* A **Weierstrass-tétel** szerint minden $[-1, 1] \to \mathbb{R}$ folytonos függvény egyenletesen, tetszőleges pontossággal megközelíthető polinomokkal.

Gondolhatnánk, hogy ennek bizonyítása lehetne az, hogy adott $n>0$ egész esetén veszünk egy $n+1$ pontból álló, egyenletes rácsot a $[-1, 1]$ intervallumon, majd ezekre $n$-edfokú polinomot illesztünk és készen vagyunk, de sajnos ez nincs így.

Nézzük meg, hogy tudjuk-e reprodukálni a problémát akkor, ha a fenti képletben elhagyjuk a $25$-öt. Hogyan magyarázható az ábrákon tapasztalt különbség?

In [ ]:
def f_fv2(x):
    return 1 / (1 + x**2)

ax = plt.axes()

for n in [2,4,5,6,12]:
    xs=np.linspace(-1,1,n)
    poly=poly_fit_newton(xs,f_fv2(xs))
    poly_v=np.vectorize(poly)
    ax.plot(xx, poly_v(xx), label="n=" + str(n))


ax.plot(xx, f_fv2(xx), label="függvény")    

ax.legend()
ax.set_xlabel(r"x")
ax.set_ylabel(r"y")

### P4. Feladat
Írjunk programot, mely egy polinomot a Horner-séma szerint értékel ki.

**Emlék:**
$$ c_0 + c_1 x + c_2 x^2 + \ldots \qquad \text{helyett} \qquad c_0 + x \left( c_1 + x \left( c_2 + \ldots \right) \right)$$

Hogyan alakul a műveletigénye a két megközelítésnek?

In [ ]:
def horner_sema(coeff, x):
    # coeff: a polinom együtthatói
    # x: a pont ahol ki szeretnénk értékelni a polinomot
    
    accumulator = 0
    for c in coeff[:0:-1]:
        accumulator += c
        accumulator *= x
    
    
    return accumulator + coeff[0]

In [ ]:
horner_sema([0,1,1,2], 3)

### P5. Feladat

Írjunk programot, ami adott $x$ és $y, y'$ -pontokat és az ott felveendő értékeket, illetve deriváltakat tartalmazó- vektorokra Hermite-féle interpolációs polinomot illeszt, majd egy további $z$ pontban kiértékeli azt.

In [ ]:
import numpy as np

def poly_fit_hermite(xs, ys, ys_):
    def w(k):
        if k == 1:
            return lambda z: 1
        return lambda z: np.prod(z - xs[0:k-1])
        
    def dw(k):
        if k == 1:
            return lambda z: 0
        A = lambda z: z * np.ones((len(xs[0:k-1]), 1)).T - xs[0:k-1]
        return lambda z: np.sum(np.prod(A(z) - np.diag(np.diag(A(z))) + np.eye(len(xs[0:k-1])), axis=1))

    def p(k):
        if k == 0:
            return lambda z: 0
        return lambda z: p(k-1)(z) + (w(k)(z))**2 * (a(k)*(z - xs[k-1]) + b(k))
        
    def dp(k):
        if k == 0:
            return lambda z: 0
        return lambda z: dp(k-1)(z) + 2*dw(k)(z)*w(k)(z)*(a(k)*(z - xs[k-1]) + b(k)) + w(k)(z)**2 * a(k)

    def b(k):
        if k == 1:
            return ys[0]
        return (ys[k-1] - p(k-1)(xs[k-1])) / (w(k)(xs[k-1]))**2

    def a(k):
        if k == 1:
            return ys_[0]
        return (ys_[k-1] - dp(k-1)(xs[k-1]) - 2*dw(k)(xs[k-1])*(w(k)(xs[k-1]))*b(k)) / (w(k)(xs[k-1]))**2
    
    n = min(min(len(xs), len(ys)), len(ys_))
    return p(n)

# Példa használat
xs = np.array([0,1,2])
ys = np.array([0,1,8])
ys_ = np.array([0,3,12])

interpolation_poly = poly_fit_hermite(xs, ys, ys_)

# Kiértékeljük az interpolációs polinomot egy adott pontban
x = 3
print("Az interpolációs polinom értéke x =", x, "pontban:", interpolation_poly(x))


### P6. Feladat

Írjunk programot, ami előállítja a Newton-alakhoz tartozó együtthatókat (pl. egy táblázat formájában).

In [ ]:
xs = np.array([-1, 0, 1])
ys = np.array([1, 0, 1])

def delta(n: int, m: int):
    assert n <= m
    
    if n == m:
        return ys[n]
    
    return (delta(n, m-1) - delta(n+1, m))/(xs[n] - xs[m])

def delta_table(xs: np.array, ys: np.array):
    assert len(xs) == len(ys)
    dX = xs.reshape((-1, 1)) - xs
    deltas = np.diag(ys) + 0.0
    
    for j in range(1, len(ys)):
        d = np.diag(deltas, j-1)
        deltas += np.diag((d[:-1] - d[1:])/np.diag(dX, j), j)
    
    return deltas

In [ ]:
# 1*1 + (-1)*(x+1) + 1*x(x+1) = x^2
delta_table(xs, ys)

In [ ]:
delta(0, 2)